In [1]:
'''
SIMPLE_CASE_STOCH.IPYNB: Stochastic simulation of a simple case of the Punisher reacting to the loss of a single synthetic gene.
'''
# By Kirill Sechkar

# PACKAGE IMPORTS 
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS
import synthetic_circuits as circuits
from cell_model import *

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


Loading BokehJS ...

In [2]:
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, circuit_v = cellmodel_auxil.add_circuit(
    circuits.punisher_xtra_initialise,
    circuits.punisher_xtra_ode,
    circuits.punisher_xtra_F_calc,
    par, init_conds,
    circuit_v=circuits.punisher_xtra_v)  # load the circuit

In [3]:
# PARAMETERISE THE CIRCUIT

# BURDENSOME SYNTHETIC GENE
par['c_xtra'] = 1
par['a_xtra'] = 1e5

# PUNISHER
# switch gene conc
par['c_switch'] = 10.0  # gene concentration (nM)
par['a_switch'] = 2000.0  # promoter strength (unitless)
par['d_switch']=0.01836
#integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch']/40.0  # RBS weaker than for the switch gene
par['d_int'] = 0.01836  # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
init_conds['cat_pb'] = 10.0  # gene concentration (nM) - INITIAL CONDITION< NOT PARAMETER as it can be cut out by the integrase
par['a_cat'] = 500.0  # promoter strength (unitless)
# synthetic protease gene
par['c_prot'] = 10.0  # gene concentration (nM)
par['a_prot'] = 50.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0 # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 700.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['baseline_switch_alt'] = 0
par['p_switch_ac_frac'] = 0.8  # active fraction of protein (i.e. share of molecules bound by the inducer)

# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 10.5 * (10.0 ** 3)

In [4]:
# SET UP SIMULATION TIME FRAMES

# deterministic simulation to reach the steadyu state
tf_det=(0,50)
# stochastic simulation before synthetic gene loss
tf_preloss=(50,55)
# stochastic simulation after synthetic gene loss
tf_afterloss=(55,70)

In [5]:
# SET UP THE DETERMINISTIC SIMULATION PARAMETERS

# diffrax simulator
savetimestep = 0.1  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver

In [6]:
# SET UP THE TAU-LEAPING SIMULATION PARAMETERS

tau = 5e-7  # simulation time step
tau_odestep = 5e-7  # number of ODE integration steps in a single tau-leap step (smaller than tau)
tau_savetimestep = 1e-2  # save time step a multiple of tau

key_seeds=[0] # random number generator seeds - NUMBER OF KEYS DEFINES NUMBER OF TRAJECTORIES

In [7]:
# RUN THE DETERMINISTIC SIMULATION TO GET THE STEADY STATE

# initial simulation to get the steady state without gene expression loss
sol=ode_sim(par,    # dictionary with model parameters
            ode_with_circuit,   #  ODE function for the cell with synthetic circuit
            cellmodel_auxil.x0_from_init_conds(init_conds,circuit_genes,circuit_miscs),  # initial condition VECTOR
            len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
            cellmodel_auxil.synth_gene_params_for_jax(par,circuit_genes), # synthetic gene parameters for calculating k values
            tf_det, jnp.arange(tf_det[0], tf_det[1], savetimestep), # time frame and time axis for saving the system's state
            rtol, atol)    # relative and absolute tolerances
ts_det=np.array(sol.ts)
xs_det=np.array(sol.ys)

In [8]:
# RUN TAU-LEAP SIMULATION

mRNA_count_scales, S, x0_tauleap, circuit_synpos2genename, keys0 = tauleap_sim_prep(par, len(circuit_genes),
                                                                                    len(circuit_miscs),
                                                                                    circuit_name2pos, xs_det[-1,:],
                                                                                    key_seeds=key_seeds)
ts_jnp_preloss, xs_jnp_preloss, final_keys_preloss = tauleap_sim(par,  # dictionary with model parameters
                                                                 circuit_v,  # circuit reaction propensity calculator
                                                                 x0_tauleap,
                                                                 # initial condition VECTOR (processed to make sure random variables are appropriate integers)
                                                                 len(circuit_genes), len(circuit_miscs),
                                                                 circuit_name2pos,
                                                                 cellmodel_auxil.synth_gene_params_for_jax(par,
                                                                                                           circuit_genes),
                                                                 # synthetic gene parameters for calculating k values
                                                                 tf_preloss, tau, tau_odestep, tau_savetimestep,
                                                                 # simulation parameters: time frame, tau leap step size, number of ode integration steps in a single tau leap step
                                                                 mRNA_count_scales, S, circuit_synpos2genename,
                                                                 # mRNA count scaling factor, stoichiometry matrix, synthetic gene number in list of synth. genes to name decoder
                                                                 keys0,
                                                                 avg_dynamics=False)  # starting random number genereation key

# concatenate the results with the deterministic simulation
ts_preloss = np.concatenate((ts_det, np.array(ts_jnp_preloss)))
xs_first_preloss = np.concatenate((xs_det, np.array(xs_jnp_preloss[1])))  # getting the results from the first random number generator key in vmap
xss_preloss = np.concatenate((xs_det * np.ones((keys0.shape[0], 1, 1)), np.array(xs_jnp_preloss)),axis=1)  # getting the results from all vmapped trajectories

# simulate after synthetic gene expression loss
par['func_prot'] = 0.0  # burdensome gene no longer present
ts_jnp, xs_jnp, final_keys = tauleap_sim(par,  # dictionary with model parameters
                                         circuit_v,  # circuit reaction propensity calculator
                                         xs_jnp_preloss[:,-1,:],
                                         # initial condition VECTOR (processed to make sure random variables are appropriate integers)
                                         len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                                         cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes),
                                         # synthetic gene parameters for calculating k values
                                         tf_afterloss, tau, tau_odestep, tau_savetimestep,
                                         # simulation parameters: time frame, tau leap step size, number of ode integration steps in a single tau leap step
                                         mRNA_count_scales, S, circuit_synpos2genename,
                                         # mRNA count scaling factor, stoichiometry matrix, synthetic gene number in list of synth. genes to name decoder
                                         keys0=final_keys_preloss,
                                         avg_dynamics=False)  # starting random number genereation key
# concatenate the results with the deterministic simulation
ts = np.concatenate((ts_preloss, np.array(ts_jnp)))
xs_first = np.concatenate((xs_first_preloss, np.array(xs_jnp[1])))  # getting the results from the first random number generator key in vmap
xss = np.concatenate((xss_preloss, np.array(xs_jnp)), axis=1)  # getting the results from all vmapped trajectories

In [9]:
# PLOT THE CELL MODEL TRAJECTORIES


# native mRNA, tRNA & protein levels, chloramphenicol level
nat_mrna_fig, nat_prot_fig, nat_trna_fig, h_fig = cellmodel_auxil.plot_native_concentrations_multiple(ts, xss, par,
                                                                                                      circuit_genes,
                                                                                                      tspan=(tf_preloss[0],tf_afterloss[-1]),
                                                                                                      simtraj_alpha=0.1)  # plot simulation results

# cell growth, translation elongation, ribosome synth. regulation, ppGpp level, tRNA aminoacylation, Resource Competition Denominator
l_figure, e_figure, Fr_figure, ppGpp_figure, nu_figure, D_figure = cellmodel_auxil.plot_phys_variables_multiple(ts, xss,
                                                                                                                par,
                                                                                                                circuit_genes,
                                                                                                                circuit_miscs,
                                                                                                                circuit_name2pos,
                                                                                                                tspan=(tf_preloss[0],tf_afterloss[-1]),
                                                                                                                simtraj_alpha=0.1)  # plot simulation results
# show plots
bkplot.show(bklayouts.grid([[nat_mrna_fig, nat_prot_fig, None],
                            [nat_trna_fig, h_fig, l_figure],
                            [e_figure, Fr_figure, D_figure]]))

In [10]:
# PLOT - SYNTHETIC GENE CIRCUIT

# synthetic mRNA and protein levels, different states of CAT gene DNA
het_mrna_fig, het_prot_fig, misc_fig = cellmodel_auxil.plot_circuit_concentrations_multiple(ts, xss, par, circuit_genes,
                                                                                            circuit_miscs,
                                                                                            circuit_name2pos,
                                                                                            circuit_styles,
                                                                                            tspan=(tf_preloss[0],tf_afterloss[-1]),
                                                                                            simtraj_alpha=0.1)  # plot simulation results

# synthetic gene regulation
F_fig = cellmodel_auxil.plot_circuit_regulation_multiple(ts, xss, par, circuit_F_calc, circuit_genes, circuit_miscs,
                                                circuit_name2pos, circuit_styles, tspan=(tf_preloss[0],tf_afterloss[-1]),
                                                simtraj_alpha=0.1)  # plot simulation results

# show plots
bkplot.show(bklayouts.grid([[het_mrna_fig, het_prot_fig, misc_fig],
                            [F_fig, None, None]]))